# 🔐 Face Verification: One-to-One Face Comparison

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **face verification** - comparing two faces to determine if they belong to the same person:

- ✅ Compare face embeddings using cosine similarity
- ✅ Set thresholds for same/different person decisions
- ✅ Batch compare multiple face pairs

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets/test_images', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["test_images/image0.jpg", "test_images/image1.jpg", "test_images/image2.jpg"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import matplotlib.pyplot as plt

import uniface
from uniface import FaceAnalyzer
from uniface.detection import RetinaFace
from uniface.recognition import ArcFace

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize Face Analyzer

We need both detection and recognition models for face verification.

In [ ]:
analyzer = FaceAnalyzer(
    detector=RetinaFace(confidence_threshold=0.5),
    recognizer=ArcFace()
)

## 4️⃣ Load and Analyze Two Faces

In [ ]:
image_path1 = 'assets/test_images/image0.jpg'
image_path2 = 'assets/test_images/image1.jpg'

image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

faces1 = analyzer.analyze(image1)
faces2 = analyzer.analyze(image2)

print(f'✓ Detected {len(faces1)} face(s) in image 1')
print(f'✓ Detected {len(faces2)} face(s) in image 2')

## 5️⃣ Display the Two Faces

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axes[0].set_title('Image 1')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axes[1].set_title('Image 2')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 6️⃣ Compute Face Similarity

Face similarity is computed using **cosine similarity** between face embeddings:
- **Range**: -1 to 1 (higher = more similar)
- **Threshold**: 0.6 is commonly used (above = same person)

In [ ]:
if faces1 and faces2:
    face1 = faces1[0]
    face2 = faces2[0]

    similarity = face1.compute_similarity(face2)
    print(f'Similarity: {similarity:.4f}')
else:
    print('⚠ Error: Could not detect faces')

## 7️⃣ Make Verification Decision

In [ ]:
THRESHOLD = 0.6

if faces1 and faces2:
    is_same_person = similarity > THRESHOLD

    print(f'📊 Similarity: {similarity:.4f}')
    print(f'📏 Threshold: {THRESHOLD}')
    print(f'🎯 Result: {"✅ Same person" if is_same_person else "❌ Different people"}')

## 8️⃣ Batch Comparison: Multiple Pairs

Compare multiple face pairs efficiently:

In [ ]:
image_pairs = [
    ('assets/test_images/image0.jpg', 'assets/test_images/image1.jpg'),
    ('assets/test_images/image0.jpg', 'assets/test_images/image2.jpg'),
    ('assets/test_images/image1.jpg', 'assets/test_images/image2.jpg'),
]

print('Comparing multiple pairs:')
print('-' * 45)
for img1_path, img2_path in image_pairs:
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    faces_a = analyzer.analyze(img1)
    faces_b = analyzer.analyze(img2)

    if faces_a and faces_b:
        sim = faces_a[0].compute_similarity(faces_b[0])
        img1_name = img1_path.split('/')[-1]
        img2_name = img2_path.split('/')[-1]
        status = "✅ Match" if sim > THRESHOLD else "❌ No match"
        print(f'{img1_name} vs {img2_name}: {sim:.4f} {status}')

---

## 📝 Summary

| Feature | Description |
|---------|-------------|
| **Similarity Range** | -1 to 1 (cosine similarity) |
| **Default Threshold** | 0.6 (same person if above) |
| **Method** | `face.compute_similarity(other_face)` |
| **Use Case** | Identity verification, access control |

### Threshold Guidelines

| Threshold | Use Case |
|-----------|----------|
| **0.5** | Lenient matching (more false positives) |
| **0.6** | Balanced (recommended default) |
| **0.7+** | Strict matching (fewer false positives) |

---

## 🔗 Next Steps

- **Face Search**: Find a person in a crowd → [04_face_search.ipynb](./04_face_search.ipynb)
- **Face Analysis**: Age, gender prediction → [05_face_analyzer.ipynb](./05_face_analyzer.ipynb)
- **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)